Here we do a symlar diagonalization analysis that we did before but using a **non-hermitian hamiltonian**. We do
this to **add dissipation** to our system and try to inder a coherence length from it, in the style of the paper https://journals.aps.org/prresearch/abstract/10.1103/PhysRevResearch.5.L042031 

**Remember to renormalize the Hamiltonian again**

**Avoid the qubit site when plugging gamma**

**The broadening shhould be smaller than the energy gaps otherwise we dont resolve anything.**


In [1]:
import sys
sys.path.append('../scripts') 

import numpy as np
import pandas as pd
# from scipy.fft import fft, ifft, fftfreq
from tqdm import tqdm


#from scipy.integrate import simpson
# from scipy.interpolate import UnivariateSpline

import qutip
from qutip_tools import *

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import h5py
import json
import matplotlib 
from mpl_toolkits.axes_grid1 import make_axes_locatable

import os
import sys  
from ast import literal_eval

In [2]:
# define basic figure sizes
bwidth=7
bheight=5
base_font = 20

matplotlib.rcParams.update({'font.size': base_font,"legend.fontsize":base_font-4 ,"font.family": "serif", 
                            "font.serif": "CMU Serif, Times New Roman",
                           'text.usetex' : True })

sns.set_palette("colorblind")

fig_route = "../plots/perturbative/"
data_route = "../data/sims/L=21/"


# Functions

In [3]:
def create_hamiltonians(L, T, Bond, Gamma):
    # creates the decoupled and interacting hamiltonians 
    
    # L_qpc = qpc lattice sites
    # T = qubit hopping
    # Gamma = dissipation
    # BOnd = index for bond locatin
    H_matrix = np.zeros((2*L,2*L)) + 0j
    # fill the diagonal with dissipation
    d_indices= kth_diag_indices(H_matrix,0)
    H_matrix[d_indices] = -1j*Gamma
    # fill in the dd hopping 
    d_indices= kth_diag_indices(H_matrix,1)
    H_matrix[d_indices] = -T 
    
    # fill in the QPC hopping
    d_indices= kth_diag_indices(H_matrix,2)
    H_matrix[d_indices] = -J[0]
    
    # when qpc and qubit hop a the same time there is no contribution
    d_indices= kth_diag_indices(H_matrix,1)
    odd_inds = (d_indices[0][1::2], d_indices[1][1::2])
    H_matrix[odd_inds] = 0
    
    # save the free hamiltonian for later use
    Hdeco = H_matrix.copy()
    
    # Fill in the interaction at the bond
    H_matrix[2*Bond,2*(Bond+1)] = H_matrix[2*Bond,2*(Bond+1)]+ Omega
    
    # Now the elemets below the diagonal
    for i in range(0,2*L):
        for j in range(i + 1, 2*L):
            H_matrix[j, i] = H_matrix[i, j]
            Hdeco[j, i] = Hdeco[i, j]
            
    return H_matrix, Hdeco

def get_spectral(Energies, Op = 10**(-5)):
    # calculates the spectral function of th hamiltonian Ham with infinithesimal Op for
    # regularization
    E_arr = np.linspace(min(Energies.real), max(Energies.real), 9000).real
    
    G_list = [] 
    for i in range(0,len(E_arr)):
        # I put the Op above purely for regularizing the result
        Gn = np.sum(1/(E_arr[i] - Energies + 1j*Op))
        G_list.append(Gn)
    # the boundaries frow way too fast because of the delta so we take them out
    
    return E_arr[1:-1] , -2*np.imag(G_list[1:-1])

# Diagonalizing

In [10]:
Omega = 0.4
J_prime = 1

L_qpc_list = [20,40,70] # qpc length
t_list = [0.02, 0.06 ,0.3,] # rabi oscillation
gamma_list = [0.002, 0.01, 0.04, 0.06, 0.1]  # add dissipation

ll_qpc = 40
print("perturbative threshold for t")
print(np.pi/(ll_qpc+1))
print("coherence threshlod for gamma")
print(1/ll_qpc)

perturbative threshold for t
0.07662421106316569
coherence threshlod for gamma
0.025


In [5]:
# create all parameter combinations
comb_array = np.array( np.meshgrid(L_qpc_list,t_list, gamma_list)).T.reshape(-1, 3) 

np.shape(comb_array)

(45, 3)

In [6]:
# for the dataset
data_dict = {"L_qpc" : [],"t":[],"gamma":[], "Omega" : [], "free_energies": [], "free_spectral" : [], "int_energies" : [], 
             "int_spectral" : [], "gamma_energies" : [], "gamma_spectral" : [] , "E_range" : []}

for simulation_index in tqdm(range(0,np.shape(comb_array)[0]), desc="Iterating Parameters"):
    parameter_array = comb_array[simulation_index,:]
    L_qpc = int(parameter_array[0])
    t = float(parameter_array[1])
    gamma = float(parameter_array[2])
    
    L = L_qpc + 2
    bond_index =  int(L_qpc/2)
    J = np.ones(L_qpc) # QPC hopping
    J[bond_index] = J_prime  

    # build the projection opeartor to the qubit-symmetric sector of the hilbert space
    Id_qpc = np.eye(L_qpc)
    plusket = np.asarray([1/np.sqrt(2),1/np.sqrt(2)])
    p_ketbra = np.outer(plusket,plusket)
    Psym = np.eye(2*L_qpc) - np.kron(Id_qpc, p_ketbra) # tensor product
    
    H_dissipative, _ = create_hamiltonians(L_qpc, t, bond_index, gamma)
    
    # Matrix withour dissipation
    H_interact, Hdeco = create_hamiltonians(L_qpc, t, bond_index, 0)
    
    # Calculate the energies and eigenvectors
    energies_gamma, vecs_gamma = np.linalg.eig(H_dissipative)
    vecs_gamma = vecs_gamma/ np.linalg.norm(vecs_gamma, axis=0)
    
    energies_int, vecs_int = np.linalg.eig(H_interact)
    vecs_int = vecs_int/ np.linalg.norm(vecs_int, axis=0)
    
    energies_free, vecs_free = np.linalg.eig(Hdeco)
    vecs_free = vecs_free/ np.linalg.norm(vecs_free, axis=0)
    
    # Sort into bands ---------------------------------------------------
    
    # sort the free evectros into bands according to their projection
    mindices, pindices = sort_by_projection(energies_free,vecs_free, Psym)
    
    # minus band
    free_energies_m = energies_free[mindices]
    free_states_m = vecs_free[:,mindices]
    # sort by magnitude
    free_energies_m, free_states_m = mag_sort(free_energies_m, free_states_m)
    
    # plus band
    free_energies_p = energies_free[pindices]
    free_states_p = vecs_free[:,pindices]
    # sort by magnitude
    free_energies_p, free_states_p = mag_sort(free_energies_p, free_states_p)
    
    
    # sort by their overlaps with the free, nondissipative case
    sorted_indices, over_matrix = sort_by_overlap_matrix(energies_int, vecs_free,vecs_int)
    sorted_e = energies_int[sorted_indices]
    sorted_vecs = vecs_int[:,sorted_indices]
    
    int_energies_m = sorted_e[mindices]
    int_states_m = sorted_vecs[:,mindices]
    int_energies_m, int_states_m = mag_sort(int_energies_m, int_states_m)
    
    int_energies_p = sorted_e[pindices]
    int_states_p = sorted_vecs[:,pindices]
    int_energies_p, int_states_p = mag_sort(int_energies_p, int_states_p)
    
    # sort by their overlaps with the free, nondissipative case
    sorted_indices, over_matrix = sort_by_overlap_matrix(energies_gamma, vecs_free,vecs_gamma)
    sorted_e = energies_gamma[sorted_indices]
    sorted_vecs = vecs_gamma[:,sorted_indices]
    gamma_energies_m = sorted_e[mindices]
    gamma_states_m = sorted_vecs[:,mindices]
    gamma_energies_m, int_states_m = mag_sort(gamma_energies_m, gamma_states_m)
    
    gamma_energies_p = sorted_e[pindices]
    gamma_states_p = sorted_vecs[:,pindices]
    gamma_energies_p, gamma_states_p = mag_sort(gamma_energies_p, gamma_states_p)
    
    e_arr, spectral_free = get_spectral( energies_free)
    _, spectral_int = get_spectral(energies_int)
    _, spectral_gamma = get_spectral(energies_gamma)

    data_dict["t"].append(t)
    data_dict["gamma"].append(gamma)
    data_dict["L_qpc"].append(L_qpc)
    data_dict["Omega"].append(Omega)

    data_dict["free_energies"].append(energies_free)
    data_dict["free_spectral"].append(spectral_free)
    data_dict["int_energies"].append(energies_int)
    data_dict["int_spectral"].append(spectral_int)
    data_dict["gamma_energies"].append(energies_gamma)
    data_dict["gamma_spectral"].append(spectral_gamma)
    data_dict["E_range"].append(e_arr)



Iterating Parameters: 100%|█████████████| 45/45 [00:15<00:00,  2.92it/s]


In [7]:

lengths = {key: len(value) for key, value in data_dict.items()}

lengths

{'L_qpc': 45,
 't': 45,
 'gamma': 45,
 'Omega': 45,
 'free_energies': 45,
 'free_spectral': 45,
 'int_energies': 45,
 'int_spectral': 45,
 'gamma_energies': 45,
 'gamma_spectral': 45,
 'E_range': 45}

# Broadening of the DOS

for each energy band

In [8]:
data_df = pd.DataFrame.from_dict(data_dict)

print(data_df["t"].unique())
print(data_df["gamma"].unique())

data_df.head()

[0.02 0.06 0.3 ]
[0.002 0.01  0.04  0.06  0.1  ]


,L_qpc,t,gamma,Omega,free_energies,free_spectral,int_energies,int_spectral,gamma_energies,gamma_spectral,E_range
0,20,0.02,0.002,0.4,"[(-1.997661652450255+0j), (-1.9576616524502557...","[101.43408693070481, 25.38422819402984, 11.294...","[(-1.9854609489814372+0j), (-1.937474730551148...","[102.68062334683017, 25.693724978372476, 11.43...","[(-1.9854609489814363-0.0020000000000000018j),...","[952.8660242207671, 837.8514311665903, 697.698...","[-1.9972176783081392, -1.9967737041660234, -1...."
1,20,0.06,0.002,0.4,"[(2.037661652450258-0j), (1.9711456115722845+0...","[97.48067614594257, 24.385201508105425, 10.843...","[(2.019294582537452-0j), (1.9703952072547386+0...","[99.2650669719019, 24.83474773245881, 11.04542...","[(2.019294582537453-0.002000000000000008j), (1...","[950.2025016063519, 832.0595817868062, 689.355...","[-2.037208788431488, -2.0367559244127187, -2.0..."
2,20,0.30,0.002,0.4,"[(-2.277661652450257+0j), (-2.211145611572284+...","[78.02667949202952, 19.516832295074803, 8.6779...","[(-2.2545794850780068+0j), (-2.209151864848279...","[79.63711183906001, 19.923847010970665, 8.8621...","[(-2.254579485078004-0.0019999999999999974j), ...","[939.0519165582695, 799.2150993960986, 640.457...","[-2.2771554491715706, -2.2766492458928838, -2...."
3,40,0.02,0.002,0.4,"[(2.0141316023674825-0j), (-2.0141316023674722...","[99.86924225090942, 25.06127275047594, 11.2056...","[(-2.007554146059243+0j), (2.007554146059237+0...","[100.63557695821424, 25.349236836915743, 11.41...","[(2.0075541460592348-0.0019999999999999883j), ...","[991.0405726991651, 877.0724327154068, 738.737...","[-2.0136839678297753, -2.0132363332920784, -2...."
4,40,0.06,0.002,0.4,"[(2.054131602367476-0j), (2.0365608475606876+0...","[96.00382477258165, 24.07907003972741, 10.7576...","[(-2.0460215620866276+0j), (-2.036388730146940...","[96.93454713951212, 24.460377449283822, 11.058...","[(-2.04602156208663-0.001999999999999998j), (-...","[995.9370041517823, 879.9712330692745, 740.669...","[-2.05367507795314, -2.05321855353879, -2.0527..."


In [9]:
data_df[cond]

NameError: name 'cond' is not defined

In [ ]:
# comparing with non dissipative case
cond = (data_df["t"]==0.06) & (data_df["gamma"]==0.02) & (data_df["L_qpc"]==40)

E_range = data_df[cond]["E_range"].values[0]
gamma_spectral = data_df[cond]["gamma_spectral"].values[0]
int_spectral = data_df[cond]["int_spectral"].values[0]


fig, ax = plt.subplots(1,2, figsize=(2*bwidth, bheight))
ax[0].plot(E_range, int_spectral , linewidth = 1,  label = r"$\gamma=0.0$" )
ax[1].plot(E_range,gamma_spectral, c="red", linewidth = 1,
           label = r"$\gamma={}$".format(0.02 ))

ax[0].set_xlabel("E")
ax[0].set_ylabel("Spectral Function")
ax[0].set_ylim(-10,300)
ax[0].legend()

ax[1].set_xlabel("E")
ax[1].set_ylim(-10,300)
ax[1].legend()



Fixed Size

In [ ]:
nrows = data_df["gamma"].nunique()
ncols = data_df["t"].nunique()

t_vals = data_df["t"].unique()
gamma_vals = data_df["gamma"].unique()

fig, ax = plt.subplots(nrows,ncols, figsize=(2.6*bwidth, 4.5*bheight))

for row_ in range(0,nrows):
    for col_ in range(0,ncols): 
        cond = (data_df["t"]==t_vals[col_]) & (data_df["gamma"]==gamma_vals[row_]) & (data_df["L_qpc"]==40)
        E_range = data_df[cond]["E_range"].values[0]
        gamma_spectral = data_df[cond]["gamma_spectral"].values[0]
                
        ax[row_, col_].plot(E_range, gamma_spectral, c="red", linewidth = 1)
        ax[row_, col_].set_title(r"$t = {},  \; \gamma = {}$".format(t_vals[col_], gamma_vals[row_]))

fig.supylabel("Spectral Function", x=0.07)
fig.supxlabel(r"$E$",x=0.515,y=0.08)

plt.savefig(fig_route+"spectral_broadening_Lqpc={}.pdf".format(40),bbox_inches='tight')

Fixed Rabi 

In [ ]:
nrows = data_df["gamma"].nunique()
ncols = data_df["L_qpc"].nunique()

L_vals = data_df["L_qpc"].unique()
gamma_vals = data_df["gamma"].unique()

fig, ax = plt.subplots(nrows,ncols, figsize=(2.6*bwidth, 4.5*bheight))

for row_ in range(0,nrows):
    for col_ in range(0,ncols): 
        cond = (data_df["t"]==0.06) & (data_df["gamma"]==gamma_vals[row_]) & (data_df["L_qpc"]==L_vals[col_])
        E_range = data_df[cond]["E_range"].values[0]
        gamma_spectral = data_df[cond]["gamma_spectral"].values[0]
                
        ax[row_, col_].plot(E_range, gamma_spectral, c="red", linewidth = 1)
        ax[row_, col_].set_title(r"$L = {},  \; \gamma = {}$".format(L_vals[col_], gamma_vals[row_]))

fig.supylabel("Spectral Function", x=0.07)
fig.supxlabel(r"$E$",x=0.515,y=0.08)

plt.savefig(fig_route+"spectral_broadening_t={}.pdf".format(0.06),bbox_inches='tight')